In [1]:
import librosa
import numpy as np
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.metrics import confusion_matrix

In [2]:
# definesc functiile necesare citirii si prelucrarii datelor de intrare
# https://medium.com/comet-ml/applyingmachinelearningtoaudioanalysis-utm-source-kdnuggets11-19-e160b069e88

def read_sounds(files, input_file):
    sounds = []
    for i in range(len(files)):
        x, sample_rate = librosa.load(input_file + '/' + input_file + '/' + files[i], res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=128).T,axis=0)
        sounds.append(mfccs)
    return np.array(sounds)

def read_data(input_file):
    files, labels = np.loadtxt(input_file + '.txt', dtype='str', delimiter=',', unpack=True)
    return read_sounds(files, input_file), np.array(labels.astype(int))

def read_test(input_file='test'):
    test_files = np.loadtxt(input_file + '.txt', dtype='str', unpack=True)
    return test_files, read_sounds(test_files, input_file)

In [3]:
# citesc datele de intrare

train_data, train_labels = read_data('train')
val_data, val_labels = read_data('validation')
test_files, test_data = read_test()

In [4]:
# declar modelul folosit si il antrenez pe datele de intrare
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

model = MLP(hidden_layer_sizes=(50,), activation='relu', alpha=1e-1, max_iter=1000)
model.fit(train_data, train_labels)

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [5]:
# scorul pe datele de antrenare

model.score(train_data, train_labels)

0.90275

In [6]:
# scorul pe datele de validare

model.score(val_data, val_labels)

0.776

In [7]:
# matricea de confuzie pentru datele de validare

predicted_val_labels = model.predict(val_data)
confusion_matrix_val = confusion_matrix(val_labels, predicted_val_labels)
print(confusion_matrix_val)

[[370 102]
 [122 406]]


In [8]:
# sparg matricea de validare

true_negative = confusion_matrix_val[0][0]
false_positive = confusion_matrix_val[0][1]
false_negative = confusion_matrix_val[1][0]
true_positive = confusion_matrix_val[1][1]

In [9]:
# calculez precizia

precision_score = true_positive / (true_positive + false_positive)
print(precision_score)

0.7992125984251969


In [10]:
# calculez recall-ul

recall_score = true_positive / (true_positive + false_negative)
print(recall_score)

0.7689393939393939


In [11]:
# calculez si salvez predictiile pentru datele de test
# https://stackoverflow.com/questions/36210977/python-numpy-savetxt-header-has-extra-character/36211002

test_labels = model.predict(test_data)
np.savetxt('submission2.txt', np.c_[test_files, test_labels], delimiter=',', header='name,label', comments='', fmt='%s')